In [15]:
import numpy as np

class QLearningBinaryClassifier:
    def __init__(self, num_features, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.num_features = num_features
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = np.zeros((2**num_features, 2)) # initialize Q table with all zeros

    def get_state(self, x):
        # Convert binary feature vector to decimal number
        state = 0
        for i, feature in enumerate(x):
            state += feature*(2**i)
        return state

    def choose_action(self, x):
        state = self.get_state(x)
        # Epsilon-greedy policy for action selection
        if np.random.uniform() < self.epsilon:
            action = np.random.choice([0, 1])
        else:
            action = np.argmax(self.q_table[state])
        return action

    def update_q_table(self, x, action, reward, next_x):
        state = self.get_state(x)
        next_state = self.get_state(next_x)
        next_action = np.argmax(self.q_table[next_state])
        # Q-learning update rule
        self.q_table[state, action] += self.alpha*(reward + self.gamma*self.q_table[next_state, next_action] - self.q_table[state, action])

    def train(self, X, y, num_episodes):
        for episode in range(num_episodes):
            total_reward = 0
            for i in range(X.shape[0]):
                x = X[i]
                action = self.choose_action(x)
                if action == y[i]:
                    reward = 1
                else:
                    reward = -1
                next_x = X[(i+1)%X.shape[0]]
                self.update_q_table(x, action, reward, next_x)
                total_reward += reward
            print("Episode:", episode+1, "Total reward:", total_reward)
            
    def predict(self, X):
        y_pred = []
        for x in X:
            state = self.get_state(x)
            action = np.argmax(self.q_table[state])
            y_pred.append(action)
        return np.array(y_pred)



In [16]:
import numpy
import pandas as pd
df=pd.read_csv("phishing.csv")
print(df)
df=df.to_numpy()

X=df[:,1:28]
y=df[:,31]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=0)
# Generate some random binary data
# Create a Q-learning binary classifier object and train it on the data
qlearn = QLearningBinaryClassifier(num_features=X_train.shape[1])
qlearn.train(X_train, y_train, num_episodes=100)

#Use the trained classifier to predict the labels of some new data
y_pred = qlearn.predict(X_test)
print("Predicted labels:", y_pred)
count=0
for i in range(len(X_test)):
    if(y_pred[i]==y_test[i]):
        count+=1
print(count/len(y_test)*100)  
      

       Index  UsingIP  LongURL  ShortURL  Symbol@  Redirecting//  \
0          0        1        1         1        1              1   
1          1        1        0         1        1              1   
2          2        1        0         1        1              1   
3          3        1        0        -1        1              1   
4          4       -1        0        -1        1             -1   
...      ...      ...      ...       ...      ...            ...   
11049  11049        1       -1         1       -1              1   
11050  11050       -1        1         1       -1             -1   
11051  11051        1       -1         1        1              1   
11052  11052       -1       -1         1        1              1   
11053  11053       -1       -1         1        1              1   

       PrefixSuffix-  SubDomains  HTTPS  DomainRegLen  ...  UsingPopupWindow  \
0                 -1           0      1            -1  ...                 1   
1                 -1   

AttributeError: module 'numpy' has no attribute 'max'